In [1]:
import pandas as pd
import requests
import numpy as np
import missingno as msno
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

Importing the dataset

In [7]:
df=pd.read_csv('85k_df.csv')
df=df.drop('Unnamed: 0',axis=1)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,8,9,15,17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Dropping missing values from df['Primary UK SIC (2007) code'] (temporary)

In [8]:
#df.dropna(subset = ["Primary UK SIC (2007) code"], inplace=True)

In [6]:
df

,Company name,Registered number,Latest accounts date,Website,E-mail address,Company status,Status date,Number of employees\nLast avail. yr,Shareholders Funds\nth GBP Last avail. yr,Total Assets\nth GBP Last avail. yr,...,"Section M: Professional, Scientific and Technical Activities",Section N: Administrative and Support Service Activities,Section O: Public Administration and Defence; Compulsory Social Security,Section P: Education,Section Q: Human Health and Social Work Activities,"Section R: Arts, Entertainment and Recreation",Section S: Other Service Activities,Section T: Activities of Households as Employers; Undifferentiated Goods- and Services-Producing Activities of Households for Own Use,Section U: Activities of Extraterritorial Organisations and Bodies,verification_True
0,EXMOTO LIMITED,03519216,2015-05-31,www.moortowngroup.com,1,1,2016-11-17,66.0,487.0,6256.0,...,0,0,0,0,0,0,0,0,0,0
1,JAEGER COMPANY LIMITED(THE),00431769,2016-02-27,www.jaeger.co.uk,1,1,2017-05-01,NaN,5295.0,9752.0,...,0,0,0,0,0,0,0,0,0,0
2,POLYCOM (UNITED KINGDOM) LTD.,03726386,2017-12-31,www.poly.com,1,1,2020-01-24,200.0,9049.0,21985.0,...,0,1,0,0,0,0,0,0,0,0
3,ARDENT MARITIME UK LIMITED,03267511,2018-12-31,www.titansalvage.com,1,1,2020-05-21,11.0,-34202.0,13542.0,...,0,0,0,0,0,0,0,0,0,0
4,BREASLEY PILLOWS LIMITED(THE),01294330,2018-04-30,www.breasley.co.uk,1,1,2020-05-01,218.0,2761.0,9956.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75189,ANGLIA STAIRLIFTS LTD,5994116,2018-11-30,www.anglia-stairlifts.co.uk,1,0,NaN,NaN,2.0,159.0,...,0,0,0,0,0,0,0,0,0,0
75190,BLUE LIZARD EQUESTRIAN LTD,5994138,2018-12-31,www.bluelizardmarketing.com,1,0,NaN,NaN,1.0,24.0,...,0,0,0,0,0,0,0,0,0,0
75191,COMMUNITY ENVIRONMENT ASSOCIATES LIMITED,5994245,2019-03-31,www.communityenvironment.org.uk,1,0,NaN,2.0,3.0,11.0,...,1,0,0,0,0,0,0,0,0,0
75192,NDS ELECTRICAL SERVICES LIMITED,5994261,2019-11-30,www.nds-electrical.com,0,0,NaN,2.0,19.0,60.0,...,0,0,0,0,0,0,0,0,0,0


Trnasforming 'Primary UK SIC (2007) code' to string and removing "." from the string

In [9]:
df['Primary UK SIC (2007) code']=df['Primary UK SIC (2007) code'].astype(str)
def allagi(value):
    place=0
    if "." in value:
        place=value.index(".")
        return value[:place]
    else:
        return value
df['Primary UK SIC (2007) code']=df['Primary UK SIC (2007) code'].map(allagi)

Built function for 'Primary UK SIC (2007) code' classification/preprocessing for one hot label encoding and apllying it to the dataset

In [10]:
def uksic(value):
    if len(value)==4:
        if value.startswith("1") or value.startswith("2") or value.startswith("3") or value.startswith("4"):
            return 'Section A: Agriculture, Forestry and Fishing'
        elif value.startswith("5") or value.startswith("6") or value.startswith("7") or value.startswith("8") or value.startswith('9'):
            return "Section B: Mining and Quarrying"
        else:
            return value
    if len(value)==5:
        if value.startswith("10") or value.startswith("11") or value.startswith("12") or value.startswith("13") or value.startswith("14") or value.startswith("15") or value.startswith("16") or value.startswith("17") or value.startswith("18") or value.startswith("19") or value.startswith("20") or value.startswith("21") or value.startswith("22") or value.startswith("23") or value.startswith("24") or value.startswith("25") or value.startswith("26") or value.startswith("27") or value.startswith("28") or value.startswith("29") or value.startswith("30") or value.startswith("31") or value.startswith("32") or value.startswith("33") or value.startswith("34"):
            return "Section C: Manufacturing"
        elif value.startswith('35'):
            return "Section D: Electricity, Gas, Steam and Air Conditioning Supply"
        elif value.startswith("36") or value.startswith("37") or value.startswith("38") or value.startswith("39") or value.startswith("40") :
            return "Section E: Water Supply; Sewerage, Waste Management and Remediation Activities"
        elif value.startswith("41") or value.startswith("42") or value.startswith("43") or value.startswith("44"):
            return "Section F: Construction"
        elif value.startswith("45") or value.startswith("46") or value.startswith("47") or value.startswith("48"):
            return "Section G: Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles"
        elif value.startswith("49") or value.startswith("50") or value.startswith("51") or value.startswith("52") or value.startswith("53") or value.startswith("54"):
            return "Section H: Transportation and Storage"
        elif  value.startswith("55") or value.startswith("56") or value.startswith("57"):
            return "Section I: Accommodation and Food Service Activities"
        elif value.startswith("58") or value.startswith("59") or value.startswith("60") or value.startswith("61") or value.startswith("62") or value.startswith("63"):
            return "Section J: Information and Communication"
        elif value.startswith("64") or value.startswith("65") or value.startswith("66") or value.startswith("67"):
            return "Section K: Financial and Insurance Activities"
        elif value.startswith("68"):
            return 'Section L: Real Estate Activities'
        elif value.startswith("69") or value.startswith("70") or value.startswith("71") or value.startswith("72") or value.startswith("73") or value.startswith("74") or value.startswith("75") or value.startswith("76"):
            return "Section M: Professional, Scientific and Technical Activities"
        elif value.startswith("77") or value.startswith("78") or value.startswith("79") or value.startswith("80") or value.startswith("81") or value.startswith("82") or value.startswith("83"):
            return "Section N: Administrative and Support Service Activities"
        elif value.startswith('84'):
            return "Section O: Public Administration and Defence; Compulsory Social Security"
        elif value.startswith("85"):
            return "Section P: Education"
        elif  value.startswith("86") or value.startswith("87") or value.startswith("88") or value.startswith("89"):
            return "Section Q: Human Health and Social Work Activities"
        elif  value.startswith("90") or value.startswith("91") or value.startswith("92") or value.startswith("93"):
            return "Section R: Arts, Entertainment and Recreation"
        elif value.startswith("94") or value.startswith("95") or value.startswith("96"):
            return "Section S: Other Service Activities"
        elif value.startswith("97") or value.startswith("98"):
            return "Section T: Activities of Households as Employers; Undifferentiated Goods- and Services-Producing Activities of Households for Own Use"
        elif value.startswith("99"):
            return "Section U: Activities of Extraterritorial Organisations and Bodies"
        else:
            return value
        
#     else:  
#         return value
df['Primary UK SIC (2007) code']=df['Primary UK SIC (2007) code'].map(uksic)

'Primary UK SIC (2007) code' missing values will be collected potentially

In [12]:
# comp=[]
# for company in open_api:
#     r=requests.get("https://api.opencorporates.com/v0.4/companies/gb/{}&api_token=qddimB3QY01UFMa0zYHo".format(str(company)))
#     df2=r.json()
#     comp.append(df2)

In [ ]:
comp

In [7]:
df['Years of Twitter account']=pd.to_datetime(df['Years of Twitter account'])

df['Months of having a Twitter account'] = pd.to_datetime('01/06/2020')-df['Years of Twitter account']
df['Months of having a Twitter account']=df['Months of having a Twitter account'].astype('timedelta64[M]')
df['Months of having a Twitter account'].astype(float)


0         96.0
1        130.0
2         99.0
3         52.0
4         58.0
         ...  
85496    124.0
85497     42.0
85498     77.0
85499    107.0
85500     11.0
Name: Months of having a Twitter account, Length: 85501, dtype: float64

Function for NaN values replacement for one hot label encoding

In [8]:
def nunclean(value):
    if pd.isnull(value) == True:
        return 0
    else:
        return 1  
for col in df[["E-mail address",'Facebook','Instagram','Youtube','LinkedIn']]:
    df[col]=df[col].map(nunclean)    

'Primary UK SIC (2007) code' dummy creation

In [9]:
#df2=pd.DataFrame()
for col in df[['Primary UK SIC (2007) code']]:
    dummies= pd.get_dummies(df[col],drop_first=True)
    df=df.merge(dummies,left_index=True,right_index=True)
    df=df.drop(col,axis=1)

'Verification' dummy creation

In [10]:
for col in df[["Verification"]]:
    dummies= pd.get_dummies(df[col],prefix='verification',drop_first=True)
    df=df.merge(dummies,left_index=True,right_index=True)
    df=df.drop(col,axis=1)

"Company status" dummy creation

In [11]:
def status(value):
    if value=="Active":
        return 0
    else:
        return 1
df['Company status']=df['Company status'].map(status)

In [12]:
def liquid(value):
    if value=='n.s.':
        return np.nan
    else:
        return value

Numerical values transformation to float

In [13]:
for i in df[['Number of employees\nLast avail. yr',
       'Shareholders Funds\nth GBP Last avail. yr',
       'Total Assets\nth GBP Last avail. yr',
       'Current Assets\nth GBP Last avail. yr',
       'Current Liabilities\nth GBP Last avail. yr',
       'Long Term Liabilities\nth GBP Last avail. yr',
       'Cost of Sales\nth GBP Last avail. yr',
       'Taxation\nth GBP Last avail. yr', 'Turnover\nth GBP Last avail. yr',
       'EBITDA\nth GBP Last avail. yr', 'EBIT margin (%)\nLast avail. yr',
       'Profit (Loss) for period [=Net income]\nth GBP Last avail. yr',
       'Liquidity Ratio (x)\nLast avail. yr',
       'Profit (Loss) before Taxation\nth GBP Last avail. yr', 'diff']]:
    try:
        df[i]=df[i].astype(str).str.replace(',', '').astype(float)
    except ValueError as e:
        continue

In [14]:
df

,Company name,Registered number,Latest accounts date,Website,E-mail address,Company status,Status date,Number of employees\nLast avail. yr,Shareholders Funds\nth GBP Last avail. yr,Total Assets\nth GBP Last avail. yr,...,"Section M: Professional, Scientific and Technical Activities",Section N: Administrative and Support Service Activities,Section O: Public Administration and Defence; Compulsory Social Security,Section P: Education,Section Q: Human Health and Social Work Activities,"Section R: Arts, Entertainment and Recreation",Section S: Other Service Activities,Section T: Activities of Households as Employers; Undifferentiated Goods- and Services-Producing Activities of Households for Own Use,Section U: Activities of Extraterritorial Organisations and Bodies,verification_True
0,EXMOTO LIMITED,03519216,2015-05-31,www.moortowngroup.com,1,1,2016-11-17,66.0,487.0,6256.0,...,0,0,0,0,0,0,0,0,0,0
1,JAEGER COMPANY LIMITED(THE),00431769,2016-02-27,www.jaeger.co.uk,1,1,2017-05-01,NaN,5295.0,9752.0,...,0,0,0,0,0,0,0,0,0,0
2,POLYCOM (UNITED KINGDOM) LTD.,03726386,2017-12-31,www.poly.com,1,1,2020-01-24,200.0,9049.0,21985.0,...,0,1,0,0,0,0,0,0,0,0
3,ARDENT MARITIME UK LIMITED,03267511,2018-12-31,www.titansalvage.com,1,1,2020-05-21,11.0,-34202.0,13542.0,...,0,0,0,0,0,0,0,0,0,0
4,BREASLEY PILLOWS LIMITED(THE),01294330,2018-04-30,www.breasley.co.uk,1,1,2020-05-01,218.0,2761.0,9956.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85496,UL ROWING LIMITED,6691735,2019-09-30,www.ulbc.co.uk,1,0,NaN,NaN,1.0,1.0,...,0,0,0,0,0,1,0,0,0,0
85497,S K MARKETING & DESIGN LIMITED,6691743,2018-09-30,www.s-kapedesign.com,1,0,NaN,1.0,0.0,12.0,...,1,0,0,0,0,0,0,0,0,0
85498,SOLENT CYCLES LIMITED,6691763,2019-09-30,www.solent-cycles.co.uk,1,0,NaN,9.0,85.0,352.0,...,0,0,0,0,0,0,0,0,0,0
85499,TOMBOY FILMS (UK) LIMITED,6691984,2018-12-31,www.tomboyfilms.co.uk,1,0,NaN,3.0,181.0,295.0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
df.to_csv( "new_85k.csv" ,index=False)